In [2]:
import pandas as pd
import numpy as np

In [46]:
# Gather all csv files with required features

path_as = "/Users/ashishsaini/TU Study/git_work"  # path to the folder where all the csv files are stored for Ashish
path_ta = "/Users/tarun/Documents/Projects/GitHub" # path to the folder where all the csv files are stored for Tarun

df_pit_stop = pd.read_csv(path_as + '/Formula-1/our-data/2020/races/01-austria/pit-stops.csv')
df_reace_results = pd.read_csv(path_as + '/Formula-1/our-data/2020/races/01-austria/race-results.csv')
df_fastest_laps = pd.read_csv(path_as + '/Formula-1/our-data/2020/races/01-austria/fastest-laps.csv')
df_driver_standings = pd.read_csv(path_as + '/Formula-1/our-data/2020/races/01-austria/driver-standings.csv')


print(df_pit_stop.columns, "\n", df_driver_standings.columns, "\n", df_fastest_laps.columns, "\n", df_reace_results.columns)

Index(['time', 'constructorId', 'engineManufacturerId', 'stop', 'driverId',
       'position', 'tyreManufacturerId', 'lap', 'driverNumber'],
      dtype='object') 
 Index(['driverId', 'position', 'points'], dtype='object') 
 Index(['time', 'constructorId', 'engineManufacturerId', 'interval', 'gap',
       'driverId', 'position', 'tyreManufacturerId', 'lap', 'driverNumber'],
      dtype='object') 
 Index(['time', 'constructorId', 'gridPosition', 'engineManufacturerId',
       'timePenalty', 'interval', 'laps', 'gap', 'driverId', 'reasonRetired',
       'position', 'points', 'tyreManufacturerId', 'driverNumber'],
      dtype='object')


In [44]:
# Renaiming and seleting required columns
df_pit_stop.rename(columns={'lap': 'pit_stop_lap'}, inplace=True)
df_pit_stop = df_pit_stop[['driverId', 'stop', 'pit_stop_lap']]

# Creating a pivot table for pit stop laps
df_pit_lap = df_pit_stop.pivot(index="driverId", columns="stop", values="pit_stop_lap")
df_pit_lap.columns = [f"pit_stop_{col}_lap" for col in df_pit_lap.columns]
df_pit_lap.reset_index(inplace=True)

df_pit_lap.head()

/var/folders/j3/kvt1lc_514sd137sknzg2hzw0000gn/T/ipykernel_5783/1522818414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pit_stop.rename(columns={'lap': 'pit_stop_lap'}, inplace=True)


,driverId,pit_stop_1_lap,pit_stop_2_lap,pit_stop_3_lap
0,alexander-albon,26.0,51.0,56.0
1,antonio-giovinazzi,25.0,55.0,56.0
2,carlos-sainz-jr,26.0,52.0,56.0
3,charles-leclerc,26.0,51.0,56.0
4,daniil-kvyat,26.0,51.0,56.0


In [47]:
df_fastest_laps["fastets_lap"] = df_fastest_laps['position'].apply(lambda x: 1 if x == 1 else 0)
df_fastest_laps = df_fastest_laps[['driverId', 'fastets_lap']]
df_fastest_laps.head()  

,driverId,fastets_lap
0,lando-norris,1
1,valtteri-bottas,0
2,lewis-hamilton,0
3,charles-leclerc,0
4,carlos-sainz-jr,0


In [53]:
df_reace_results = df_reace_results[['gridPosition','driverId', 'constructorId']]
df_reace_results.head()

,gridPosition,driverId,constructorId
0,1,valtteri-bottas,mercedes
1,7,charles-leclerc,ferrari
2,3,lando-norris,mclaren
3,5,lewis-hamilton,mercedes
4,8,carlos-sainz-jr,mclaren


In [51]:
df_driver_standings

,driverId,position,points
0,valtteri-bottas,1,25
1,charles-leclerc,2,18
2,lando-norris,3,16
3,lewis-hamilton,4,12
4,carlos-sainz-jr,5,10
5,sergio-perez,6,8
6,pierre-gasly,7,6
7,esteban-ocon,8,4
8,antonio-giovinazzi,9,2
9,sebastian-vettel,10,1


In [54]:
df_reace_results.shape, df_fastest_laps.shape, df_pit_lap.shape, df_driver_standings.shape

((20, 3), (20, 2), (16, 4), (13, 3))

In [58]:
df_mer_1 = pd.merge(df_reace_results, df_fastest_laps, how='inner', on='driverId')
df_mer_2 = pd.merge(df_mer_1, df_pit_lap, how='left', on='driverId')
df_mer_3 = pd.merge(df_mer_2, df_driver_standings, how='left', on='driverId')

In [59]:
df_mer_3

,gridPosition,driverId,constructorId,fastets_lap,pit_stop_1_lap,pit_stop_2_lap,pit_stop_3_lap,position,points
0,1,valtteri-bottas,mercedes,0,26.0,56.0,NaN,1.0,25.0
1,7,charles-leclerc,ferrari,0,26.0,51.0,56.0,2.0,18.0
2,3,lando-norris,mclaren,1,26.0,51.0,56.0,3.0,16.0
3,5,lewis-hamilton,mercedes,0,26.0,56.0,NaN,4.0,12.0
4,8,carlos-sainz-jr,mclaren,0,26.0,52.0,56.0,5.0,10.0
5,6,sergio-perez,racing-point,0,26.0,56.0,NaN,6.0,8.0
6,12,pierre-gasly,alphatauri,0,26.0,56.0,NaN,7.0,6.0
7,14,esteban-ocon,renault,0,26.0,56.0,NaN,8.0,4.0
8,18,antonio-giovinazzi,alfa-romeo,0,25.0,55.0,56.0,9.0,2.0
9,11,sebastian-vettel,ferrari,0,26.0,51.0,56.0,10.0,1.0


In [47]:
df_mer_3.rename(columns={'position': 'final_position'}, inplace=True)
df_mer_3.head()

,gridPosition,driverId,fastets_lap,stop,constructorId,pit_stop_lap_position,pit_stop_lap,final_position,points
0,1,valtteri-bottas,0,1,mercedes,5,26,1,25
1,1,valtteri-bottas,0,2,mercedes,26,56,1,25
2,7,charles-leclerc,0,1,ferrari,10,26,2,18
3,7,charles-leclerc,0,2,ferrari,19,51,2,18
4,7,charles-leclerc,0,3,ferrari,31,56,2,18


In [60]:
df_mer_3

,gridPosition,driverId,constructorId,fastets_lap,pit_stop_1_lap,pit_stop_2_lap,pit_stop_3_lap,position,points
0,1,valtteri-bottas,mercedes,0,26.0,56.0,NaN,1.0,25.0
1,7,charles-leclerc,ferrari,0,26.0,51.0,56.0,2.0,18.0
2,3,lando-norris,mclaren,1,26.0,51.0,56.0,3.0,16.0
3,5,lewis-hamilton,mercedes,0,26.0,56.0,NaN,4.0,12.0
4,8,carlos-sainz-jr,mclaren,0,26.0,52.0,56.0,5.0,10.0
5,6,sergio-perez,racing-point,0,26.0,56.0,NaN,6.0,8.0
6,12,pierre-gasly,alphatauri,0,26.0,56.0,NaN,7.0,6.0
7,14,esteban-ocon,renault,0,26.0,56.0,NaN,8.0,4.0
8,18,antonio-giovinazzi,alfa-romeo,0,25.0,55.0,56.0,9.0,2.0
9,11,sebastian-vettel,ferrari,0,26.0,51.0,56.0,10.0,1.0


In [ ]:
pd.merge??

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'MergeHow' = 'inner',
    on: 'IndexLabel | AnyArrayLike | None' = None,
    left_on: 'IndexLabel | AnyArrayLike | None' = None,
    right_on: 'IndexLabel | AnyArrayLike | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool | None' = None,
    indicator: 'str | bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning:

In [ ]:
# old codes

'''max_stops = df_pit_stop['stop'].max()

merged_dfs = {}
for stop in range(0, max_stops):
    merged_dfs[stop] = df_pit_stop[df_pit_stop['stop'] == stop+1]
    merged_dfs[stop].rename(columns={'pit_stop_lap': 'pit_stop_'+str(stop+1)+'_lap'}, inplace=True)
    merged_dfs[stop].rename(columns={'pit_stop_lap_position': 'pit_stop_'+str(stop+1)+'_lap_position'}, inplace=True)
'''




# for stop in range(0, len(merged_dfs)):
#     merged_dfs[stop]['pit_stop_'+str(stop+1)] = merged_dfs[stop]['pit_stop_lap']

# for stop in range (1, len(merged_dfs)):
#     merged_dfs[stop] = merged_dfs[stop][['driverId', 'pit_stop_'+str(stop+1)]]

# final_df = merged_dfs[0][['driverId']]
# for stop in range (0, len(merged_dfs)):
#     if stop != len(merged_dfs)-1:
#         merged_df_new = pd.merge( merged_dfs[stop],  merged_dfs[stop+1], on='driverId', how='inner')
#         final_df = pd.merge( final_df,  merged_df_new, on='driverId', how='inner')


# # merged_df_new = pd.merge( merged_dfs[0],  merged_dfs[1], on='driverId', how='inner')

# final_df

"""def merge_fun(merged_dfs):

    merged_1 = merged_dfs[0]
    for i in range (1, len(merged_dfs)):
        merged_1 = pd.merge(merged_1, merged_dfs[i], on = ["driverId"], how = "left")

    return merged_1[['driverId', 'pit_stop_1_lap', 'pit_stop_2_lap', 'pit_stop_3_lap']]
"""